In [2]:
%matplotlib inline
from __future__ import division

import pandas as pd
import matplotlib.pyplot as plt
import utils
import datetime
import numpy as np
from collections import Counter

In [3]:
visitors, events, devices, url_categories = utils.load_data(event_categories=True)

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5,6,7,8,11,12,13,14,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [7]:
specialties = visitors.groupby('primary_specialty').npi_number.count().sort_values(ascending=False).reset_index()
total = specialties.npi_number.sum()
specialties['pct'] = specialties.npi_number.apply(lambda x: x/total)
specialties.to_csv('../data/specialty_breakdown.csv', index=False)

### Personas

In [4]:
# https://en.wikipedia.org/wiki/Ternary_plot
    
def to_cartesian(a,b,c):
    x = .5*(2*b+c)/(a+b+c)
    y = (np.sqrt(3)/2)*(c/(a+b+c))
    return (x,y)

def get_persona(x,y):
    if y > np.sqrt(3)/4:
        return 'butterfly'
    elif y + np.sqrt(3)*x <= np.sqrt(3)/2:
        return 'unicorn'
    elif np.sqrt(3)*x - y >= np.sqrt(3)/2:
        return 'bookworm'
    else:
        return 'cat?'
    
def get_personas(event_sessions):
    personas = event_sessions.groupby('dg_id').use_case.apply(list).reset_index()
    personas = pd.merge(personas, visitors[['dg_id', 'primary_specialty']], on='dg_id')
    personas['total_sessions'] = personas.use_case.apply(lambda x: len(x))
    personas['use_case_counts'] = personas.use_case.apply(lambda x: Counter(x))
    
    # parse out use case counts
    personas['pharma'] = personas.use_case_counts.apply(lambda x: x['pharma_device'])
    personas['publications'] = personas.use_case_counts.apply(lambda x: x['publications_ed_tools'])
    personas['social'] = personas.use_case_counts.apply(lambda x: x['professional_social_media'])
    personas['total_3d'] = personas.apply(lambda x: x.pharma + x.publications + x.social, axis=1)
    
    # exclude folks with only 'other' category (will address eventually)
    personas = personas[personas.total_3d > 0]
    
    # normalize per user
    personas['pharma_pct'] = personas.apply(lambda x: x.pharma/x.total_3d, axis=1)
    personas['pubs_pct'] = personas.apply(lambda x: x.publications/x.total_3d, axis=1)
    personas['social_pct'] = personas.apply(lambda x: x.social/x.total_3d, axis=1)
    
    # get coordinates to define persona
    personas['ternary_coordinates'] = personas.apply(lambda x: to_cartesian(x.pharma_pct, x.pubs_pct, x.social_pct), axis=1)
    personas['persona'] = personas.ternary_coordinates.apply(lambda x: get_persona(x[0], x[1]))
    personas['ternary_x'] = personas.ternary_coordinates.apply(lambda x: x[0])
    personas['ternary_y'] = personas.ternary_coordinates.apply(lambda x: x[1])
    
    return personas

In [1]:
personas

NameError: name 'personas' is not defined

### Session Level Summary Statistics

In [7]:
def get_quarters(dt):
    if dt < datetime.date(2017, 5, 1):
        return 'q1'
    elif dt < datetime.date(2017, 8, 1):
        return 'q2'
    
def get_use_case(events, three=True):
    if three:
        publisher_tools = ['Publisher', 'Reference Tool', 'Medical Education', 'Multi-Channel Marketing']
        pharma = ['Pharma', 'Med Device']
        social = ['Professional Social', 'Medical Association', 'Recruiter']

        events['use_case'] = ''
        events.loc[events.site_category.isin(publisher_tools), 'use_case'] = 'publications_ed_tools'
        events.loc[events.site_category.isin(pharma), 'use_case'] = 'pharma_device'
        events.loc[events.site_category.isin(social), 'use_case'] = 'professional_social_media'
        events.loc[events.use_case=='', 'use_case'] = 'other'
    else:
        use_cases = {
            'publication_research' : ['Publisher'],
            'education_tools' : ['Reference Tool', 'Medical Education', 'Multi-Channel Marketing'],
            'pharma' : ['Pharma', 'Med Device'],
            'social_professional' : ['Professional Social', 'Medical Association', 'Recruiter']
        }

        for u in use_cases:
            events.loc[events.site_category.isin(use_cases[u]), 'use_case'] = u
        events.loc[events.use_case=='', 'use_case'] = 'other'
    return events

def aggregate_sessions(events, output_file=None, use_case_truncated=False, personas=False):
    ev = get_use_case(events, three=use_case_truncated)
    ev['quarter'] = ev.timestamp.apply(lambda x: get_quarters(x.date()))
    sessions = ev.groupby('session_id').event_id.count().reset_index()
    sessions = sessions.rename(columns={'event_id':'page_views'})
    event_sessions = events.drop_duplicates('session_id')
    event_sessions = pd.merge(event_sessions, sessions, on='session_id')
    event_sessions = pd.merge(visitors, event_sessions, on='dg_id')
    
    tableau_cols = ['timestamp', 'dg_id', 'npi_number', 'primary_specialty', 'primary_specialty_group', 
                    'site_category', 'site_sub_category', 'disease_category', 
                    'disease', 'pharma_firm', 'use_case', 'page_views']
    if personas:
        personas = get_personas(event_sessions)
        event_sessions = pd.merge(event_sessions, personas[['dg_id', 'persona']], on='dg_id')
        tableau_cols.append('persona')
    
    if output_file:
        tableau_sessions = event_sessions[tableau_cols]
        tableau_sessions.to_csv(output_file, index=False)
    else:
        return event_sessions

In [8]:
aggregate_sessions(events, use_case_truncated=True, personas=True, output_file='../data/sessions_with_personas_specialty_groups.csv')

In [5]:
event_sessions = pd.read_csv('../data/sessions_with_personas_specialty_groups.csv')

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
event_sessions.head()

,timestamp,dg_id,npi_number,primary_specialty,primary_specialty_group,site_category,site_sub_category,disease_category,disease,pharma_firm,use_case,page_views,persona
0,2017-02-04 12:44:07.660,R0000000026,NaN,NaN,NaN,Multi-Channel Marketing,Multi Topic,Multiple,NaN,NaN,publications_ed_tools,1,bookworm
1,2017-03-17 13:20:51.610,R0000000098,NaN,NaN,NaN,Publisher,Multi Topic,Multiple,NaN,NaN,publications_ed_tools,1,bookworm
2,2017-04-09 12:11:33.693,R0000000358,NaN,NaN,NaN,Recruiter,Multi Topic,Multiple,NaN,NaN,professional_social_media,1,butterfly
3,2017-05-03 15:40:07.033,R0000000651,NaN,NaN,NaN,Publisher,Multi Topic,Multiple,NaN,NaN,publications_ed_tools,2,bookworm
4,2017-05-05 19:35:55.610,R0000000651,NaN,NaN,NaN,Publisher,Multi Topic,Multiple,NaN,NaN,publications_ed_tools,1,bookworm


In [7]:
personas = get_personas(event_sessions)

In [9]:
personas.groupby('persona')[['pharma_pct', 'pubs_pct', 'social_pct']].min()

,pharma_pct,pubs_pct,social_pct
persona,,,
bookworm,0.000000,0.500000,0.000000
butterfly,0.000000,0.000000,0.526316
cat?,0.066667,0.111111,0.027778
unicorn,0.500000,0.000000,0.000000


In [18]:
personas[['dg_id', 'primary_specialty', 'persona', 'ternary_x', 'ternary_y']].to_csv('../data/persona_coordinates.csv')

### NPI Level Summary Statistics

In [6]:
def categorize_frequency(n):
    if n < 1:
        return 'Less than 1 visit a month' 
    elif n <= 10:
        return '1-10 times per month'
    else:
        return 'More than 10x per month'
    
def npi_level_summary_statistics(output_file=None, npis=None, personas=False):
    # if given a list of users, pull in those. otherwise, use all available data
    if npis:
        v = visitors[visitors.npi_number.isin(npis)]
    else:
        v = visitors[visitors.exclude==0]
    ev = pd.merge(v, events, on='dg_id')
    if len(ev) == 0:
        print 'target list has no web activity'
        return
    # check which users have web activity
    event_dgids = set(ev.dg_id)
    v['has_event'] = v.dg_id.apply(lambda x: 1 if x in event_dgids else 0)
    
    # check frequency of web activity (timeframe denominator starts with the earliest date the user has activity)
    max_dt = max(events.timestamp).date()
    ev['dt'] = ev.timestamp.apply(lambda x: x.date())
    ev['quarter'] = ev.dt.apply(lambda x: get_quarters(x))
    freq = ev.drop_duplicates(subset=['dg_id', 'dt']).groupby(['dg_id']).dt.agg(['min', 'max', 'count']).reset_index()
    freq['sessions_per_month'] = freq.apply(lambda x: x['count']/(max(1, (max_dt-x['min']).days)/30), axis=1)
    freq['frequency_category'] = freq.sessions_per_month.apply(lambda x: categorize_frequency(x))
    v = pd.merge(v, freq[['dg_id', 'sessions_per_month', 'frequency_category']], how='left')
    
    # separate frequency by quarter
    freqq = ev.drop_duplicates(subset=['dg_id', 'dt']).groupby(['dg_id', 'quarter']).dt.agg(['min', 'max', 'count']).reset_index()
    freqq['sessions_per_month'] = freqq.apply(lambda x: x['count']/(max(1, (max_dt-x['min']).days)/30), axis=1)
    freqq['frequency_category'] = freqq.sessions_per_month.apply(lambda x: categorize_frequency(x))
    freqq = freqq.pivot(index='dg_id', columns='quarter', values='frequency_category').reset_index()
    freqq.columns= ['dg_id', 'frequency_category_q1', 'frequency_category_q2']
    v = pd.merge(v, freqq, on='dg_id', how='left')
    
    # separate frequency by month
    ev['month'] = ev.timestamp.apply(lambda x: x.strftime('%B'))
    ev['year'] = ev.timestamp.apply(lambda x: x.year)

    freqm = ev.drop_duplicates(subset=['dg_id', 'dt'])\
        .groupby(['dg_id', 'month', 'year']).dt.agg(['min', 'max', 'count'])\
        .reset_index().rename(columns={'count':'sessions_per_month'})

    freqm['frequency_category'] = freqm.sessions_per_month.apply(lambda x: categorize_frequency(x))
    freqm = freqm.pivot(index='dg_id', columns='month', values='frequency_category').reset_index()
    freqm = freqm.fillna('Less than 1 visit a month')
    freqm.columns = ['dg_id'] + ['frequency_category_{}'.format(x.lower()) for x in freqm.columns[1:]]
    v = pd.merge(v, freqm, on='dg_id', how='left')

    # grab avg number of urls per session
    urls_per_session = ev.groupby(['dg_id', 'session_id']).url.nunique().reset_index()\
        .groupby('dg_id').url.mean().reset_index()
    urls_per_session = urls_per_session.rename(columns={'url':'urls_per_session'})
    v = pd.merge(v, urls_per_session, on='dg_id', how='left')
    
    tableau_columns = [
        'dg_id', 
        'identity_type', 
        'professional_designation', 
        'npi_number', 
        'primary_specialty', 
        'primary_specialty_group', 
        'birth_year', 
        'grad_year', 
        'gender', 
        'has_event', 
        'sessions_per_month',
        'urls_per_session' 
    ]
    
    tableau_columns+=[x for x in v.columns if 'frequency' in x]
        
    if personas:
        event_sessions = aggregate_sessions(ev, use_case_truncated=True)
        personas = get_personas(event_sessions)
        v = pd.merge(v, personas[['dg_id', 'persona']], on='dg_id', how='left')
        tableau_columns.append('persona')
        
    if output_file:
        v[tableau_columns].to_csv(output_file, index=False)
    else:
        return v[tableau_columns]

In [59]:
# generate new sample of NPIs for dashboard
npis = list(visitors[(~visitors.npi_number.isnull())\
                     &(visitors.professional_designation=='Physician')\
                     &(visitors.exclude==0)]\
            .sample(10000).npi_number)

npi_level_summary_statistics(output_file='../data/npi_sample_with_personas_20170821.csv', npis=npis, personas=True)
#df = npi_level_summary_statistics(npis=npis, personas=True)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
npi_level_summary_statistics(output_file='../data/npi_full_with_personas_20170821.csv', personas=True)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
